# NCAA Tournament

## Imports

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

from selenium import webdriver
from selenium.webdriver import Chrome
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests

import copy
import random

import multiprocessing
from joblib import Parallel, delayed

## Functions

In [3]:
#Webdriver parser used to extract the brackets
def parse2(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(url)
    sleep(3)
    sourceCode = driver.page_source
    return sourceCode

In [4]:
def score_bracket(bracket, results):
    points = 0
    for key in bracket.keys():
        if key != "points":
            bracket[key] = set(bracket[key])
            results[key] = set(results[key])
            points += rounds_points[key] * len(bracket[key] & results[key])
    return points

In [5]:
def comp_brackets(results, brackets, round_index=0, index=0):
	bracket_scores = [[0 for x in range(len(brackets))] for y in range(len(brackets))]
	if results["1"][0] != '':
		ordered = []
		results_copy = results.copy()
		for b in range(len(brackets)):
			bracket = brackets[b]
			score = score_bracket(bracket, results_copy)
			ordered.append((b, score))
		ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
		for x in range(len(ordered)):
			bracket_scores[ordered[x][0]][x] = 1
		return bracket_scores
	else:
		if index == int(rounds[round_index]) - 1:
			next_round_index = round_index + 1
			next_index = 0
		else:
			next_round_index = round_index
			next_index = index + 1


		if results[rounds[round_index]][index] == '':
			options = [results[rounds[round_index - 1]][index * 2], results[rounds[round_index - 1]][index * 2 + 1]]
			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[0]
			branch1 = comp_brackets(results_copy, brackets, next_round_index, next_index)

			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[1]
			branch2 = comp_brackets(results_copy, brackets, next_round_index, next_index)
			total_branch = np.add(branch1, branch2)
			return total_branch
		else:
			return comp_brackets(results, brackets, next_round_index, next_index)

def comp_brackets_weighted(results, brackets, team_to_seed, round_index=0, index=0, liklihood = 1):
	bracket_scores = [[0 for x in range(len(brackets))] for y in range(len(brackets))]
	if results["1"][0] != '':
		ordered = []
		results_copy = results.copy()
		for b in range(len(brackets)):
			bracket = brackets[b]
			score = score_bracket(bracket, results_copy)
			ordered.append((b, score))
		ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
		for x in range(len(ordered)):
			bracket_scores[ordered[x][0]][x] = 1 * liklihood
		return bracket_scores
	else:
		if index == int(rounds[round_index]) - 1:
			next_round_index = round_index + 1
			next_index = 0
		else:
			next_round_index = round_index
			next_index = index + 1


		if results[rounds[round_index]][index] == '':
			options = [results[rounds[round_index - 1]][index * 2], results[rounds[round_index - 1]][index * 2 + 1]]
			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[0]
			update = seed_vs_seed[team_to_seed[options[0]]][team_to_seed[options[1]]]
			branch1 = comp_brackets_weighted(results_copy, brackets, team_to_seed, next_round_index, next_index, liklihood * update)

			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[1]
			update = seed_vs_seed[team_to_seed[options[1]]][team_to_seed[options[0]]]
			branch2 = comp_brackets_weighted(results_copy, brackets, team_to_seed, next_round_index, next_index, liklihood * update)
			total_branch = np.add(branch1, branch2)
			return total_branch
		else:
			return comp_brackets_weighted(results, brackets, team_to_seed, next_round_index, next_index, liklihood)

In [68]:
def comp_MC(results, brackets, time_allocated = 10, samples = 1000, std_thresh = 0.1):
    start_time = time.time()
    trials = []
    while time.time() - start_time < time_allocated:
        bracket_scores = [np.array([0 for x in range(len(brackets))]) for y in range(len(brackets))]
        for trial in range(samples):
            results_copy = copy.deepcopy(results)
            for round_index in range(len(results_copy)-1):
                round = rounds[round_index]
                for index in range(len(results_copy[round])):
                    if results_copy[round][index] == '':
                        options = [results_copy[rounds[round_index - 1]][index * 2], results_copy[rounds[round_index - 1]][index * 2 + 1]]
                        option = options[random.choice([0,1])]
                        results_copy[round][index] = option
            ordered = []
            for b in range(len(brackets)):
                bracket = brackets[b]
                score = score_bracket(bracket, results_copy)
                ordered.append((b, score))
            ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
            for x in range(len(ordered)):
                bracket_scores[ordered[x][0]][x] += 1
        trials.append(np.array(bracket_scores) / samples)

        if len(trials) > 10:
            std = np.array(trials)
            test = std[0,:,0]
            max_win = np.argmax(test)
            std = std[:, max_win, 0]
            std = np.std(std) * 100
            if std < std_thresh:
                break
    trials = np.array(trials)
    std = trials.std(axis=0)
    trials = trials.mean(axis=0)
    return trials, std

def comp_MC_weighted(results, brackets, team_to_seed, time_allocated = 10, samples = 1000, std_thresh = 0.1):
    start_time = time.time()
    trials = []
    while time.time() - start_time < time_allocated:
        bracket_scores = [np.array([0 for x in range(len(brackets))]) for y in range(len(brackets))]
        for trial in range(samples):
            results_copy = copy.deepcopy(results)
            for round_index in range(len(results_copy)-1):
                round = rounds[round_index]
                for index in range(len(results_copy[round])):
                    if results_copy[round][index] == '':
                        options = [results_copy[rounds[round_index - 1]][index * 2], results_copy[rounds[round_index - 1]][index * 2 + 1]]
                        weights = [seed_vs_seed[team_to_seed[options[0]]][team_to_seed[options[1]]], seed_vs_seed[team_to_seed[options[1]]][team_to_seed[options[0]]]]
                        option = options[random.choices([0,1], weights=weights)[0]]
                        results_copy[round][index] = option
            ordered = []
            for b in range(len(brackets)):
                bracket = brackets[b]
                score = score_bracket(bracket, results_copy)
                ordered.append((b, score))
            ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
            for x in range(len(ordered)):
                bracket_scores[ordered[x][0]][x] += 1
        trials.append(np.array(bracket_scores) / samples)
        if len(trials) > 10:
            std = np.array(trials)
            test = std[0,:,0]
            max_win = np.argmax(test)
            std = std[:, max_win, 0]
            std = np.std(std) * 100
            if std < std_thresh:
                break
    trials = np.array(trials)
    std = trials.std(axis=0)
    trials = trials.mean(axis=0)
    return trials, std

In [7]:
#Gets the actual games and results that have been played
def get_so_far(url):
    print(url)
    bracket = {"64": [], "32": [], "16": [], "8": [], "4": [], "2": [], "1": []}
    data = requests.get(url)
    soup = BeautifulSoup(data.text, features='html.parser')
    index = 0
    team_to_seed = {}
    for x in range(1, 64):
        class_i = "matchup m_" + str(x)
        mydivs = soup.find_all("div", {"class": class_i})
        if mydivs == []:
            class_ii = class_i + " userPickable"
            mydivs = soup.find_all("div", {"class": class_ii})
            if mydivs == []:
                class_ii = class_i + " homeOnBottom" + " userPickable"
                mydivs = soup.find_all("div", {"class": class_ii})
            mydivs = mydivs[0].find_all("span", {"class": "name"})
            team1 = mydivs[0].text.strip()
            team2 = mydivs[1].text.strip()
            team3 = mydivs[2].text.strip()
            team4 = mydivs[3].text.strip()
        else:
            mydivs = mydivs[0].find_all("div", {"class": "slots"})
            team1 = mydivs[0].find_all("div", {"class": "slot s_1"})
            team1 = team1[0].find_all("span", {"class": "name"})
            team1 = team1[0].text.strip()

            team2 = mydivs[0].find_all("div", {"class": "slot s_2"})
            team2 = team2[0].find_all("span", {"class": "name"})
            team2 = team2[0].text.strip()
        if x <= 32:
            bracket["64"].append(team1)
            bracket["64"].append(team2)
            team_to_seed[team1] = seed_order[index]
            team_to_seed[team2] = seed_order[index + 1]
            index += 2
            if index == 16:
                index = 0
        elif x <= 48:
            bracket["32"].append(team1)
            bracket["32"].append(team3)
        elif x <= 56:
            bracket["16"].append(team1)
            bracket["16"].append(team3)
        elif x <= 60:
            bracket["8"].append(team1)
            bracket["8"].append(team3)
        elif x <= 62:
            bracket["4"].append(team1)
            bracket["4"].append(team3)
        elif x <= 63:
            bracket["2"].append(team1)
            bracket["2"].append(team3)

    mydivs = soup.find_all("div", {"class": "slot userPickable"})
    if len(mydivs) == 0:
        mydivs = soup.find_all("span", {"class": "actual winner"})
    teams = mydivs[0].find_all("span", {"class": "name"})
    team1 = teams[0].text.strip()
    # team2 = teams[1].text.strip()
    bracket["1"].append(team1)
    bracket["points"] = 1920
    return bracket, team_to_seed

#Gets the picks for a bracket for each game
def get_bracket(url):
    bracket = {"64": [], "32": [], "16": [], "8": [], "4": [], "2": [], "1": [], "points": 0}
    data = requests.get(url)
    soup = BeautifulSoup(data.text, features='html.parser')
    classes = []
    for x in range(1, 64):
        class_i = "matchup m_" + str(x)

        mydivs = soup.find_all("div", {"class": class_i})

        if mydivs == []:
            class_ii = class_i + " userPickable"
            mydivs = soup.find_all("div", {"class": class_ii})

            if mydivs == []:
                class_ii = class_i + " homeOnBottom" + " userPickable"
                mydivs = soup.find_all("div", {"class": class_ii})
            try:
                mydivs = mydivs[0].find_all("span", {"class": "name"})
            except:
                return {'64': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '32': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '16': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '8': ['1', '1', '1', '1', '1', '1', '1', '1'], '4': ['1', '1', '1', '1'], '2': ['1', '1'],
                        '1': ['1'], 'points': 260}

            team1 = mydivs[0].text.strip()
            team2 = mydivs[1].text.strip()
            team3 = mydivs[2].text.strip()
            team4 = mydivs[3].text.strip()
        else:
            mydivs = mydivs[0].find_all("div", {"class": "slots"})
            team1 = mydivs[0].find_all("div", {"class": "slot s_1"})
            team1 = team1[0].find_all("span", {"class": "name"})
            team1 = team1[0].text.strip()

            team2 = mydivs[0].find_all("div", {"class": "slot s_2"})
            team2 = team2[0].find_all("span", {"class": "name"})
            team2 = team2[0].text.strip()

        if x <= 32:
            bracket["64"].append(team1)
            bracket["64"].append(team2)
        elif x <= 48:
            bracket["32"].append(team2)
            bracket["32"].append(team4)
            if team1 == team2:
                bracket["points"] += 10
            if team3 == team4:
                bracket["points"] += 10
        elif x <= 56:
            bracket["16"].append(team2)
            bracket["16"].append(team4)
            if team1 == team2:
                bracket["points"] += 20
            if team3 == team4:
                bracket["points"] += 20
        elif x <= 60:
            bracket["8"].append(team2)
            bracket["8"].append(team4)
            if team1 == team2:
                bracket["points"] += 40
            if team3 == team4:
                bracket["points"] += 40
        elif x <= 62:
            bracket["4"].append(team2)
            bracket["4"].append(team4)
            if team1 == team2:
                bracket["points"] += 80
            if team3 == team4:
                bracket["points"] += 80
        elif x <= 63:
            bracket["2"].append(team2)
            bracket["2"].append(team4)
            if team1 == team2:
                bracket["points"] += 160
            if team3 == team4:
                bracket["points"] += 160
    try:
        mydivs = soup.find_all("div", {"class": "slot userPickable"})
        teams = mydivs[0].find_all("span", {"class": "name"})
    except:
        try:
            mydivs = soup.find_all("div", {"class": "slot wWinner userPickable"})
            teams = mydivs[0].find_all("span", {"class": "name"})
        except:
            mydivs = soup.find_all("div", {"class": "slot s_2 correct wWinner userPickable"})
            teams = mydivs[0].find_all("span", {"class": "name"})
    team1 = teams[0].text.strip()
    team2 = teams[1].text.strip()
    bracket["1"].append(team2)
    if team1 == team2:
        bracket["points"] += 320

    return bracket

#Gets the IDs of all the participants in a group
def get_group(group):
    IDs = []
    soup = BeautifulSoup(parse2(group))
    mydivs = soup.find_all("table", {"class": "type_entries"})
    entries = mydivs[0].find_all("a", {"class": "entry"})
    for entry in entries:
        IDs.append((entry.get('href'), entry.text.strip()))
    return IDs

## Constants

In [8]:
base_bracket_url = "https://fantasy.espn.com/tournament-challenge-bracket/2022/en/"

rounds = ["64", "32", "16", "8", "4", "2", "1"]

rounds_points = {"64": 0, "32": 10, "16": 20, "8": 40, "4": 80, "2": 160, "1": 320}

seed_vs_seed = {'1': {'1': 0.5, '2': 0.539, '3': 0.634, '4': 0.711, '5': 0.839, '6': 0.706, '7': 0.857, '8': 0.798, '9': 0.901, '10': 0.857, '11': 0.556, '12': 1.0, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.993}, '2': {'1': 0.461, '2': 0.5, '3': 0.603, '4': 0.444, '5': 0.167, '6': 0.722, '7': 0.694, '8': 0.444, '9': 0.5, '10': 0.645, '11': 0.833, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.938, '16': 0.5}, '3': {'1': 0.366, '2': 0.397, '3': 0.5, '4': 0.625, '5': 0.5, '6': 0.576, '7': 0.611, '8': 0.5, '9': 0.5, '10': 0.692, '11': 0.679, '12': 0.5, '13': 0.5, '14': 0.847, '15': 0.5, '16': 0.5}, '4': {'1': 0.289, '2': 0.556, '3': 0.375, '4': 0.5, '5': 0.563, '6': 0.333, '7': 0.333, '8': 0.364, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.69, '13': 0.79, '14': 0.5, '15': 0.5, '16': 0.5}, '5': {'1': 0.161, '2': 0.833, '3': 0.5, '4': 0.438, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.671, '13': 0.842, '14': 0.5, '15': 0.5, '16': 0.5}, '6': {'1': 0.294, '2': 0.278, '3': 0.424, '4': 0.667, '5': 0.5, '6': 0.5, '7': 0.625, '8': 0.5, '9': 0.5, '10': 0.6, '11': 0.634, '12': 0.5, '13': 0.5, '14': 0.875, '15': 0.5, '16': 0.5}, '7': {'1': 0.143, '2': 0.306, '3': 0.389, '4': 0.667, '5': 0.5, '6': 0.333, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.601, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '8': {'1': 0.202, '2': 0.556, '3': 0.5, '4': 0.636, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.518, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '9': {'1': 0.099, '2': 0.5, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.482, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '10': {'1': 0.143, '2': 0.355, '3': 0.308, '4': 0.5, '5': 0.5, '6': 0.4, '7': 0.399, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '11': {'1': 0.444, '2': 0.167, '3': 0.321, '4': 0.5, '5': 0.5, '6': 0.366, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.9, '15': 0.5, '16': 0.5}, '12': {'1': 0.1, '2': 0.5, '3': 0.5, '4': 0.31, '5': 0.329, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.75, '14': 0.5, '15': 0.5, '16': 0.5}, '13': {'1': 0.5, '2': 0.5, '3': 0.5, '4': 0.21, '5': 0.158, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.25, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '14': {'1': 0.5, '2': 0.5, '3': 0.153, '4': 0.5, '5': 0.5, '6': 0.125, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.1, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '15': {'1': 0.5, '2': 0.063, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '16': {'1': 0.007, '2': 0.5, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}}

seed_order = ['1', '16', "8", "9", "5", "12", "4", "13", "6", "11", "3", "14", "7", "10", "2", "15"]

## Main

In [9]:
dad_group = "https://fantasy.espn.com/tournament-challenge-bracket/2022/en/group?groupID=4452645"

IDs = get_group(dad_group)

results, team_to_seed = get_so_far(base_bracket_url + IDs[0][0])

brackets = []
for x in tqdm(range(len(IDs))):
    bracket = get_bracket(base_bracket_url + IDs[x][0])
    brackets.append(bracket)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\aidan\.wdm\drivers\chromedriver\win32\111.0.5563.64]
C:\Users\aidan\AppData\Local\Temp\ipykernel_19444\4167765881.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


https://fantasy.espn.com/tournament-challenge-bracket/2022/en/entry?entryID=54161783


100%|██████████| 13/13 [00:11<00:00,  1.11it/s]


In [43]:
# results['32'] = ['' for x in range(32)]
# results['16'] = ['' for x in range(16)]
results['8'] = ['' for x in range(8)]
results['4'] = ['', '','','']
results['2'] = ['','']
results['1'] = ['']

In [201]:
# total = comp_brackets(results, brackets)
# total = comp_MC(results, brackets)

# total = comp_brackets_weighted(results, brackets, team_to_seed)
# total = comp_MC_weighted(results, brackets, team_to_seed, time_allocated = 1)
# total = total / np.sum(total[0]) * 100

In [70]:
total, std = comp_MC(results, brackets, time_allocated=10, std_thresh=10)
# total = total / np.sum(total[0]) * 100
# columns = ["1st", "2nd", "3rd"]
# for x in range(3, len(IDs)):
# 	columns.append(str(x+1)+"th")
# rows = [entry[1] for entry in IDs]

# df = pd.DataFrame(total, columns = columns, index = rows)
# df = df.sort_values('1st', ascending=False)
# df

In [44]:
total = comp_brackets(results, brackets)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th
Brown>Tatum,18.176270,11.456299,6.787109,4.418945,3.918457,4.730225,5.706787,10.461426,15.570068,11.914062,6.176758,0.683594,0.000000
No Speak Hablo,12.304688,11.425781,9.375000,8.227539,10.235596,9.643555,10.229492,9.484863,8.343506,9.362793,1.367188,0.000000,0.000000
JMGilly 1,11.816406,15.551758,10.455322,5.157471,3.948975,4.010010,3.759766,8.154297,11.071777,9.436035,7.580566,8.276367,0.781250
sanersully1 2,11.737061,5.126953,3.070068,1.409912,1.800537,2.008057,2.368164,5.017090,6.536865,7.556152,14.013672,23.535156,15.820312
Ameliasg97 1,11.041260,15.063477,14.416504,15.429688,13.525391,9.667969,8.203125,4.809570,3.887939,2.758789,1.196289,0.000000,0.000000
PaulPierceisGOAT,10.217285,2.624512,6.109619,2.496338,3.466797,4.595947,10.345459,8.496094,5.749512,7.580566,13.513184,9.960938,14.843750
smgilson 1,8.001709,13.641357,13.989258,14.996338,11.334229,10.296631,9.277344,8.282471,6.542969,3.100586,0.537109,0.000000,0.000000
doihavetopickcolgate,7.708740,10.821533,12.097168,13.861084,12.939453,12.640381,9.783936,11.956787,4.968262,2.539062,0.683594,0.000000,0.000000
Cgilly61 1,5.541992,9.393311,13.183594,17.181396,17.169189,15.228271,10.528564,6.439209,3.613281,0.549316,1.171875,0.000000,0.000000
KMG2022,1.940918,1.367188,2.343750,2.124023,3.515625,6.134033,7.061768,8.038330,11.138916,7.800293,19.116211,29.418945,0.000000


In [11]:
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)

In [12]:
df

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,11th,12th,13th
Brown>Tatum,21.734328,9.816144,4.672108,5.062740,3.913695,4.126198,6.022221,12.602799,14.575541,12.004262,5.392716,0.077247,0.000000
Cgilly61 1,13.344141,12.867179,17.754989,17.534750,17.307127,10.879977,5.103220,4.089024,0.994001,0.035836,0.089757,0.000000,0.000000
PaulPierceisGOAT,12.782398,3.141478,7.765286,1.763735,2.495167,5.170293,12.231457,11.396208,7.111630,4.573218,14.197264,6.766392,10.605473
Ameliasg97 1,11.962040,19.483358,16.569111,16.034201,12.659978,9.801371,7.563429,2.308834,2.074475,1.341720,0.201484,0.000000,0.000000
smgilson 1,10.053334,12.538017,13.512992,18.034441,12.915198,12.713083,8.744498,5.662768,4.847490,0.956697,0.021482,0.000000,0.000000
No Speak Hablo,6.757390,6.486634,5.988939,4.179523,10.602931,12.647705,14.721734,12.222583,15.402116,10.673905,0.316541,0.000000,0.000000
doihavetopickcolgate,6.391179,8.011600,9.459075,14.911080,12.022341,14.025339,9.457202,16.873573,5.282312,3.344655,0.221645,0.000000,0.000000
sanersully1 2,6.285589,2.772306,1.289072,0.981702,1.350938,1.562365,1.591164,3.750862,6.453045,12.228314,20.533018,26.988659,14.212968
JMGilly 1,5.677967,13.801213,7.015020,2.547308,4.257008,3.813907,2.927697,9.190123,14.739841,9.916954,11.169887,12.995048,1.948027
William,3.961474,10.196442,14.204970,16.272558,14.022528,13.626561,14.674597,4.276560,2.517737,1.591215,3.741064,0.793891,0.120401
